In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import random
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from datetime import datetime

In [5]:
chrome = webdriver.Chrome(ChromeDriverManager().install())
chrome.get("https://web.whatsapp.com")



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/home/jean-christophe/.wdm/drivers/chromedriver/linux64/94.0.4606.113/chromedriver] found in cache


In [6]:
#Recherche le chemin passé en arument et le renvoie
#Si rien n'est trouvé avant le timeout, une exception est levée.
def chercher_element(chemin: str, timeout: int):
    trouve = False
    temps = 0
    while(not trouve):
        try: 
            element = chrome.find_element_by_xpath(chemin)
            trouve = True
        except: 
            time.sleep(0.5)
            temps += 0.5
        if(temps > timeout):
            raise Exception("Élément non trouvé pour le chemin : " + chemin) 
    return element

#Recherche tous les chemins passés en arument et renvoie le premier élément trouvé, ainsi que l'index du chemin auquel il correspond.
#Si rien n'est trouvé avant le timeout, une exception est levée.
#On n'utilise pas la fonction chercher_element afin de faire toutes les recherches en parallèle pour ne pas attendre les timeout à chaque fois.
def chercher_elements(chemins, timeout: int):
    trouve = False
    temps = 0
    element = None
    index = -1
    while(temps < timeout):
        for i in range(len(chemins)):
            try: 
                index = i
                element = chrome.find_element_by_xpath(chemins[i])
                trouve = True
                return (element, index)
            except:
                time.sleep(0.3)
                temps += 0.3
    raise Exception("Élément non trouvé pour les chemins : " + str(chemins))
    

def taper_message(tel: str, nom:str, prenom: str, message: str, num_contact: str, mots_cle_interdits = []):
    
    #taper le message dans le champ message
    #Le caractère '\n' est interprété comme un appui sur la touche "entrer". Ce qui envoie le message. On remplace tous les \n par une combinaison Shift + Enter
    search_box = chercher_element('/html/body/div[1]/div/div/div[4]/div/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div[@class="_13NKt copyable-text selectable-text"][@contenteditable="true"][@dir="ltr"][@spellcheck="true"]', 3)
    
    
    for ligne in message.split('\n'):
        search_box.send_keys(ligne)
        ActionChains(chrome).key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(Keys.ENTER).perform()
    return True
    

#Envoie un message au numéro de téléphone passé en argument, si celui ci est trouvé.
#La chaine de la PJ doit être un chemin absolu vers un fichier présent en local sur l'ordinateur.
#Pour ne pas mettre de PJ, simplement passer une chaine vide, ou bien ne pas préciser cet argument.
#mots_cle_interdits est un tableau de chaines de caractère. S'il n'est pas vide, le message ne sera envoyé que si la conversation sélectionnée ne contient aucun des mots clé interdits. (permet d'éviter les messages en doublon, mais prend du temps)
def envoyer_message(tel: str, nom: str, prenom: str, message: str, image='', mots_cles_interdits = []):
    global statut_envoi
    
    #On commence par chercher si un des mots-clé interdits existe dans la conversation.
    time.sleep(1) #On laisse le temps à la conversation de se charger. Sinon, la recherche n'a pas accès aux anciens messages.
    try:
        doit_envoyer = True
        for mot_cle in mots_cles_interdits :
            if(chercher_contenu_dans_discussion(mot_cle)):
                doit_envoyer = False
                statut_envoi += " | le mot-clé interdit '" + mot_cle + "' a été trouvé -> pas d'envoi"
                return False
        if(doit_envoyer):
            statut_envoi += " | aucun des mots-clés interdits n'a été trouvé -> envoi"
            print(statut_envoi)
    except Exception as e:
        print(str(e))
        statut_envoi += " | exception lors de la recherche des mots-clés"
        print(statut_envoi)
        raise Exception("exception lors de la recherche des mots-clés")
    
    #Aucun des mots-clé interdits n'a été trouvé, on continue
    if(not taper_message(tel, nom, prenom, message, num_contact, mots_cles_interdits)):
        return False
    time.sleep(0.5)
    
    if(image == ""):
        bouton_envoyer = chercher_element("//button[span/@data-testid='send']", 1)
        bouton_envoyer.click()
    else:
        #cliquer sur le bouton joindre
        boutonJoindre = chercher_element("//div[@role='button'][@title='Joindre']", 1)
        boutonJoindre.click()

        #sélection du fichier à envoyer
        input_box = chrome.find_element_by_tag_name('input')
        input_box.send_keys(image)

        #Attendre que le fichier se charge
        bouton_envoyer = chercher_element("//div[@role='button'][span/@data-testid='send']", 35)
        time.sleep(1)
        bouton_envoyer.click()
    
    statut_envoi += "| envoyé"
    return True


#Recherche la chaine passée en argument dans l'historique des message du contact actuellement sélectionné.
#Renoie vrai si au moins un message contient la chaine, faux sinon.
def chercher_contenu_dans_discussion(contenu: str):
    global statut_envoi
    try:
        bouton_chercher = chercher_element('//div[@role="button"][@title="Recherche..."]', 2)
        bouton_chercher.click()
    except:
        print()
    try:
        champ_saisie = chercher_element('//header[//*[contains(text(),"Rechercher des messages")]]/following-sibling::*//div[@contenteditable="true"][@class="_13NKt copyable-text selectable-text"]', 3)
        champ_saisie.clear()
        time.sleep(0.25)
        champ_saisie.send_keys(contenu)
        time.sleep(0.5)
    except Exception as e:
        statut_envoi += " | echec de la sélection de la barre de recherche"
        print(statut_envoi)
        raise Exception("echec de la sélection de la barre de recherche : " + str(e)) 
    
    chemin_succes = '/html/body/div[1]/div/div/div[2]/div[3]/span/div/div/div[2]/div[1]/div/div/div/div/div/div[2]/div[2]/div[1]/span/span/strong[@class="i0jNr"][text()="' + contenu + '"]'
    chemin_succes2 ='//header[//*[contains(text(),"Rechercher des messages")]]/following-sibling::div//div[@class="_3uIPm WYyr1"]/div[@class="_3m_Xw"]/div/div/div/div/div/span[@class="Hy9nV"][contains(@title, "' + contenu + '")]'
    chemin_echec  = '//header[//*[contains(text(),"Rechercher des messages")]]/following-sibling::div[@id="pane-side"]/div/div/span[@dir="auto"][@class="i0jNr"][text()="Aucun message trouvé"]'

    index = -1
    (resultat, index) = chercher_elements([chemin_succes, chemin_succes2, chemin_echec], 10)
    if(index == 0 or index == 1):
        statut_envoi += " | message vraiment trouvé"
        return True
    elif((index == 2)):
        statut_envoi += " | message vraiment non trouvé"
        return False
    else:
        statut_envoi += " | valeur index = " + str(index) + "non acceptée dans la recherche de message"
        return True
    champ_saisie.clear()

    #Si rien n'a été renvoyé à ce moment, c'est qu'il y a eu un problème -> Levée d'exception.
    statut_envoi += " | échec de la recherche"
    raise Exception("Échec de la recherche pour le contenu " + contenu) 


    
#Recherche, puis sélectionne le contact s'il est trouvé. S'il n'est pas trouvé, renvoie Faux
#Si le contact est trouvé, vérifie qu'aucun des messages passés ne contient un des mot_clés interdits (pour éviter les envois en doublon)
#Au final, ne renvoie vrai que si le contact a été correctement sélectionné et qu'il faut envoyer le message.
def selectionner_contact(tel: str, nom: str, prenom: str, num_contact: str, mots_cles_interdits = []):
    global statut_envoi
    #On commence par cliquer sur le bouton pour annuler la recherche précédente
    try:
        bouton_annuler_recherche = chrome.find_element_by_xpath('//div[@class="_3yWey XKmj6"]//button[@class="_3GYfN"]')
        bouton_annuler_recherche.click()
        #print("recherche annulée avec succès !")
        time.sleep(0.5)
    except Exception as e:
        print(str(e))
    #On recherche le champ de saisie de la recherche de contact, pour y taper le numéro de téléphone.
    try:
        search_box = chrome.find_element_by_xpath('/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[@class="_13NKt copyable-text selectable-text"][@contenteditable="true"][@data-tab="3"][@dir="ltr"][not(@spellcheck)][@title="Champ de recherche"][@role="textbox"]')
        search_box.clear()
        search_box.send_keys(tel)
    except Exception as e:
        print(str(e))
    time.sleep(0.5)
    #Xpath1 ne comporte pas nativement de méthode pour passer en casse majuscule. On fait ça à la main.
    #Maintenant qu'on a tapé le numéro de téléphone dans le champ de recherche, on cherche si un titre de conversation apparaît
    #Si le message "Aucun contact, discussion ou message trouvé" apparaît, cela signifie que cette personne n'a pas whatsapp.
    try:
        titre_UpCass = 'translate(@title,"abcdefghijklmnopqrstuvwxyzèéêëïîäâàüûùöô","ABCDEFGHIJKLMNOPQRSTUVWXYZÈÉÊËÏÎÄÂÀÜÛÙÖÔ")'
        #chaine_chemin_contact = '//div[@aria-label="Résultats de la recherche."]/div[//span[contains(' + titre_UpCass + ',"{0}") or contains(' + titre_UpCass + ',"{1}")]]'
        chaine_chemin_contact = '//div[@aria-label="Résultats de la recherche."]/*[//*[contains(' + titre_UpCass + ',"{0}") or contains(' + titre_UpCass + ',"{1}")]]'
        chaine_chemin_contact2 = '//div[@aria-label="Résultats de la recherche."]/div/div[//span[contains(' + titre_UpCass + ',"{0}") or contains(' + titre_UpCass + ',"{1}")]]'
        chaine_chemin_contact3 = '//div[@aria-label="Résultats de la recherche."]/div/div/div[//span[contains(' + titre_UpCass + ',"{0}") or contains(' + titre_UpCass + ',"{1}")]]'
        chaine_chemin_contact4 = '//div[@aria-label="Résultats de la recherche."]/div/div/div/div[//span[contains(' + titre_UpCass + ',"{0}") or contains(' + titre_UpCass + ',"{1}")]]'
        
        xpath_contact_trouve = chaine_chemin_contact.format(nom.upper(), prenom.upper())
        xpath_contact_trouve2 = chaine_chemin_contact2.format(nom.upper(), prenom.upper())
        xpath_contact_trouve3 = chaine_chemin_contact3.format(nom.upper(), prenom.upper())
        xpath_contact_trouve4 = chaine_chemin_contact4.format(nom.upper(), prenom.upper())
        xpath_contact_non_trouve = '//span[@dir="auto"][@class="i0jNr"][contains(text(),"Aucun contact, discussion ou message trouvé")]'
    except Exception as e:
        print(str(e))
    print(0)
    try:
        index = -1
        (bouton_contact, index) = chercher_elements([xpath_contact_trouve, xpath_contact_non_trouve], 3)
        print(1)
        if(index == 1):
            statut_envoi += " | Contact officiellement non trouvé"
            print("contact officiellement non trouvé")
            raise Exception("contact officiellement non trouvé")
        elif((index == -1)):
            statut_envoi += "échec de la recherche du contact"
            print("échec de la recherche du contact")
            raise Exception("échec de la recherche du contact")
        print(2)
        try:
            xpath_contact_trouves = [chaine_chemin_contact.format(nom.upper(), prenom.upper()),
                                     chaine_chemin_contact2.format(nom.upper(), prenom.upper()),
                                     chaine_chemin_contact3.format(nom.upper(), prenom.upper()),
                                     chaine_chemin_contact4.format(nom.upper(), prenom.upper())]
            bouton_contacts = [chercher_element(xpath_contact_trouve, 1),
                              chercher_element(xpath_contact_trouve2, 1),
                              chercher_element(xpath_contact_trouve3, 1),
                              chercher_element(xpath_contact_trouve4, 1)]

        except Exception as e:
            print("erreur lors de la recherche des boutons alternatifs : " + str(e))
        print(3)
        #Il semble que l'élément cliquable ne soit pas le même d'une conversation à l'autre. Je clique donc sur tout ce que je peux en espérant trouver le bon.
        temps_max = 3
        temps = 0       
        trouve = False
        
        prenom = prenom.upper().replace(" ", "").replace("-", "").replace("'", "")
        nom = nom.upper().replace(" ", "").replace("-", "").replace("'", "")
        print(4)
        while(not(trouve) and (temps < temps_max)):
            print("dans le while recherche de la conversation")
            for num_bouton in range(len(bouton_contacts)):
                try:
                    print("click sur le bouton " + str(num_bouton) + "   temps = " + str(temps))
                    bouton_contacts[num_bouton].click()
                    time.sleep(0.5)
                    Titre_conversation = chercher_element("//header/div/div/div/span[@dir='auto']", 4).text
                    Titre_conversation = Titre_conversation.upper().replace(" ", "").replace("-", "").replace("'", "")
                    trouve = (prenom in Titre_conversation) or (nom in Titre_conversation) or (str(tel) in Titre_conversation) or (("N" + num_contact) in Titre_conversation)
                    print("Titre_conversation" + Titre_conversation)
                    print("prenom : " + prenom)
                    print("prenom in Titre_conversation : " + str(prenom in Titre_conversation))
                    print("nom : " + nom)
                    print("nom in Titre_conversation : " + str(nom in Titre_conversation))
                    print("tel : " + str(tel))
                    print("str(tel) in Titre_conversation)" + str(str(tel) in Titre_conversation))
                    print("('N' + num_contact) in Titre_conversation : " + str((("N" + num_contact) in Titre_conversation)))
                    if(trouve):
                        print("Le titre correspond !")
                        break #On sort de la boucle for et le while va sortir en même temps.
                except Exception as e:
                    print(str(e))
            temps = temps + 1
        #print("après le while trouvé = " + str(trouve))
        print(5)
        search_box.clear()
        statut_envoi += " | contact trouvé 0"
        
        Titre_conversation = chercher_element("//header/div/div/div/span[@dir='auto']", 4).text

        print("Titre conversation : " + Titre_conversation)
        statut_envoi += " | 2"
        Titre_conversation = Titre_conversation.upper().replace(" ", "").replace("-", "").replace("'", "")
        statut_envoi += " | 2.1"
        if((prenom in Titre_conversation) or (nom in Titre_conversation) or (tel in Titre_conversation) or (("N" + num_contact) in Titre_conversation)):
            statut_envoi += " | contact sélectionné"
        else:
            print("Mauvais titre")
            statut_envoi += " | mais mauvais titre : '" + nom + " " + prenom + "' titre conversation : '" + Titre_conversation + "'"
            print(statut_envoi)
            raise Exception("mauvais titre : '" + nom + " " + prenom + "' titre conversation : '" + Titre_conversation + "'")

    except Exception as e:
        print(str(e))
        statut_envoi += " | Contact non trouvé"
        print(statut_envoi)
        raise Exception("Contact non trouvé")

    
    
def generer_message(prenom, nom, lycee, ville, annee):
    if(lycee != ""):
        lycee = "au lycée " + lycee
    else : 
        lycee = "dans ton lycée"
        
    #message = "Bonjour {0} {1},\n\n" + \
    #          "J'espère que tout s'e"
    #message = message.format(prenom, nom, lycee, ville)
        
    if(annee == "2020"):
        message = "Bonjour {0} {1},\n\n" + \
                  "Le premier concours d'entrée à l'IFNTI aura lieu *ce mercredi 29 juin*. Il ne reste que quelques jours pour s'inscrire.\n\n" + \
                  "Il y aura une autre session après les résultats du BAC. Mais ce sera plus difficile car il restera moins de places disponibles...\n\n" + \
                  "Cliquer ici pour s'inscrire : *https://forms.gle/9Cvqz7qpobzFgQuN7*"
        
        message = message.format(prenom, nom)
        
    elif(annee == "2021"):
        message = "Bonjour {0} {1},\n\n" + \
                  "Le premier concours d'entrée à l'IFNTI aura lieu *ce mercredi 29 juin*. Il ne reste que quelques jours pour s'inscrire.\n\n" + \
                  "Il y aura une autre session après les résultats du BAC. Mais ce sera plus difficile car il restera moins de places disponibles...\n\n" + \
                  "Cliquer ici pour s'inscrire : *https://forms.gle/9Cvqz7qpobzFgQuN7*"
        message = message.format(prenom, nom, lycee, ville)
        
    elif(annee == "2022"):
        message = "Bonjour {0} {1},\n\n" + \
                  "J'espère que tout s'est bien passé pour les épreuves du BAC2.\n\n" + \
                  "Le premier concours d'entrée à l'IFNTI aura lieu *ce mercredi 29 juin*. Il ne reste que quelques jours pour s'inscrire.\n\n" + \
                  "Il y aura une autre session après les résultats du BAC. Mais ce sera plus difficile car il restera moins de places disponibles...\n\n" + \
                  "Cliquer ici pour s'inscrire : *https://forms.gle/9Cvqz7qpobzFgQuN7*"
        message = message.format(prenom, nom, lycee, ville)
    else:
        print("erreur : étudiant avec année différent de 2020, 2021 ou 2022 : '" + str(annee) +"'")
        message = ""
    return message


statut_envoi = ""
fichier_etudiants = '/home/jean-christophe/git_IFNTI/Documentations_outils/envoi_whatsapp_supbot/liste_etudiants_interesses.csv'
fichier_log = '/home/jean-christophe/git_IFNTI/Documentations_outils/envoi_whatsapp_supbot/log_envoi.csv'
with open(fichier_etudiants, "r") as filin:
    with open(fichier_log, 'a') as filout:
        num_contact_envoye = 0
        for ligne in filin:
            statut_envoi = ""
            ligne = ligne.split(",")
            num_contact = ligne[0].replace("\n", "")
            ville = ligne[1].replace("\n", "")
            lycee = ligne[2].replace("\n", "")
            nom = ligne[3].replace("\n", "")
            prenom = ligne[4].replace("\n", "")
            prefixe_tel = ligne[5].replace("\n", "")
            tel = ligne[6].replace("-", "").replace("\n", "")
            tel2 = ligne[7].replace("-", "").replace("\n", "")
            annee = ligne[9].replace("\n", "")

            
            
            piece_jointe = ""
            #piece_jointe = "/home/jean-christophe/git_IFNTI/Administration/Outils_de_com/Videos/Presentation_IFNTI_juin_2021/Presentation_IFNTI_juin_2021_VLQ.mp4"
            #piece_jointe = "/home/jean-christophe/Images/vidage_telephone/2022/05-mai/20220521_130250.jpg"
            #piece_jointe = ""
            now = datetime.now().strftime("%d/%m/%Y,%H:%M:%S")
            
            #Pour ne pas mettre de pièce jointe, simplement mettre une chaine vide
            for num_tel in {tel, tel2}:
                if(num_tel != ""):
                    print(num_tel)
                    statut_envoi = ""
                    try:
                        selectionner_contact(num_tel, nom, prenom, num_contact)
                        message0 = generer_message(prenom, nom, lycee, ville, annee)
                        envoyer_message(num_tel, nom, prenom, message0, piece_jointe, ["Le premier concours d'entrée"])
                        message1 = "Vous cherchez une formation supérieure en informatique ? Découvrez l'IFNTI à Sokodé !\n\n" + \
                                  "*Présentation générale*\n" + \
                                  "L'objectif de l'Institut de Formation aux Normes et Technologies de l'Informatique est de faire en sorte que les étudiants togolais et plus particulièrement de la région centrale puissent bénéficier à moindre coût d'une formation supérieure en informatique, au standard international. Installé à Sokodé au cœur du Togo, l'IFNTI recrute les bacheliers scientifiques pour une licence professionnelle LMD en 3 ans.\n" + \
                                  "L'IFNTI est situé entre le lycée Technique et le lycée Moderne, dans le quartier administratif.\n\n" + \
                                  "*Conditions d'enseignement*\n" + \
                                  "La qualité de la formation est liée au taux d‘encadrement, avec de petits effectifs (promotions de 24 étudiants), et aux enseignants permanents de niveau ingénieur. Ceux-ci sont toujours disponibles pour soutenir les apprenants. Les modalités didactiques employées sont très proches de l’activité professionnelle. Depuis l'ouverture de l'IFNTI en 2014, tous les diplômés travaillent.\n\n" + \
                                  "*Candidature à l'IFNTI*\n" + \
                                  "Les promotions étant limitées à 24 étudiants, l'entrée à l'IFNTI se fait sur concours." + \
                                  "L'inscription se fait en ligne, en remplissant ce formulaire : https://forms.gle/9Cvqz7qpobzFgQuN7\n" + \
                                  "Il se déroule en deux étapes :\n" + \
                                  " - des épreuves écrites pendant une journée.\n" + \
                                  " - un entretien oral, conditionné par la réussite des épreuves écrites.\n" + \
                                  "Les deux étapes se dérouleront dans nos locaux à Sokodé.\n" + \
                                  "Vous serez évalués sur trois matières :\n" + \
                                  "- Français\n" + \
                                  "- Anglais\n" + \
                                  "- Mathématiques\n" + \
                                  "Frais d'inscription au concours : 10.000 FCFA.\n" + \
                                  "Date du premier *concours : 29 juin*. Une autre session du concours aura lieu après les résultats du bac2.\n\n" + \
                                  "*Inscription académique*\n" + \
                                  "Si vous êtes retenus après les épreuves orales, vous pourrez commencer la procédure d'inscription académique.\n" + \
                                  "Frais de scolarité : 590.000 FCFA / an.\n" + \
                                  "Frais d'inscription : 30.000 FCFA / an.\n" + \
                                  "Ces frais restent les mêmes pendant toute la formation. Les frais d'inscription ne sont pas remboursables.\n\n" + \
                                  "*Hébergement*\n" + \
                                  "L'IFNTI ne dispose pas de logements propres. Toutefois il est très facile de trouver un logement à Sokodé, et à un faible coût.\n\n" + \
                                  "Site internet\n" + \
                                  "https://www.ifnti.com\n\n" + \
                                  "Whatsapp\n" + \
                                  "+228 90 91 81 41\n\n" + \
                                  "Email\n" + \
                                  "contact@ifnti.com\n\n" + \
                                  "Vidéo de présentation\n" + \
                                  "https://www.youtube.com/watch?v=oDosebpBEf8"
                        #envoyer_message(num_tel, nom, prenom, message1, piece_jointe, ["Vous cherchez une formation"])
                        #message2 = "Voici la vidéo de présentation de l'IFNTI."
                        #piece_jointe = "/home/jean-christophe/git_IFNTI/Administration/Outils_de_com/Videos/Presentation_IFNTI_juin_2021/Presentation_IFNTI_juin_2021_VLQ.mp4"
                        #envoyer_message(num_tel, nom, prenom, message2, piece_jointe, ["Voici la vidéo de"])
                        print("\n\n\n")
                    except Exception as e:
                        print(str(e))
                    filout.write(now + "," + num_contact + "," + ville + "," + lycee + "," + nom + "," + prenom + "," + prefixe_tel + "," + num_tel + ",,," + annee + "," + statut_envoi + ",\n")
                    #Le fichier de Log a été organisé en suivant la structure du fichier liste_etudiants_intéresses.csv, de façon à pouvoir relancer le script sur tous ceux pour qui l'envoi a échoué afin de tenter une deuième fois.
                        
filout.close() 
filin.close()
print("Terminé.")

93406326
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="_3yWey XKmj6"]//button[@class="_3GYfN"]"}
  (Session info: chrome=94.0.4606.81)

0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
90170911
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
91919213
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
91595751
0
1
2
3
4
dans le while recherche de la conversation
click sur le bouton 0   temps = 0
Titre_conversationYAHYAESSOFA,2022N2028
prenom : KIDISSELE
prenom in Titre_conversation : False
nom : SOUBOR
nom in Titre_conversation : False
tel : 91595751
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : False
click su

Titre_conversationNIKABAUJEANWAGBE,2022N2035
prenom : RAÏME
prenom in Titre_conversation : False
nom : NAKOUDJA
nom in Titre_conversation : False
tel : 92924595
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : False
click sur le bouton 2   temps = 0
Titre_conversationRAÏMENAKOUDJA,2022N2039
prenom : RAÏME
prenom in Titre_conversation : True
nom : NAKOUDJA
nom in Titre_conversation : True
tel : 92924595
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : True
Le titre correspond !
5
Titre conversation : RAÏME NAKOUDJA, 2022 N2039
 | contact trouvé 0 | 2 | 2.1 | contact sélectionné | message vraiment non trouvé | aucun des mots-clés interdits n'a été trouvé -> envoi




70853765
0
1
2
3
4
dans le while recherche de la conversation
click sur le bouton 0   temps = 0
Titre_conversationRAÏMENAKOUDJA,2022N2039
prenom : TCHIKAKPAFAURE
prenom in Titre_conversation : False
nom : KAGMAN
nom in Titre_conversation : False
tel : 70853765
s

Titre_conversationMAMADOUISSA,2022N2045
prenom : AMBALI
prenom in Titre_conversation : False
nom : AGOULOU
nom in Titre_conversation : False
tel : 92390183
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : False
click sur le bouton 2   temps = 0
Titre_conversationAMBALIAGOULOU,2022N2047
prenom : AMBALI
prenom in Titre_conversation : True
nom : AGOULOU
nom in Titre_conversation : True
tel : 92390183
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : True
Le titre correspond !
5
Titre conversation : AMBALI AGOULOU, 2022 N2047
 | contact trouvé 0 | 2 | 2.1 | contact sélectionné | message vraiment non trouvé | aucun des mots-clés interdits n'a été trouvé -> envoi




90205073
0
1
2
3
4
dans le while recherche de la conversation
click sur le bouton 0   temps = 0
Titre_conversationAMBALIAGOULOU,2022N2047
prenom : RAFIOU
prenom in Titre_conversation : False
nom : BOUKARI
nom in Titre_conversation : False
tel : 90205073
str(tel) in T

0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
91645087
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
91158954
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
93706745
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
70524050
0
1
2
3
4
dans le while recherche de la conversation
click sur le bouton 0   temps = 0
Titre_conversationBADADESIREGBARRE,2022N2043
prenom : RAZAK
prenom in Titre_conversation : False
nom : IDRISSOU
nom in Titre_conversation : False
tel : 70524050
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : False
click sur le bouton 1   temps = 0

0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
93410897
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
70111816
0
1
2
3
4
dans le while recherche de la conversation
click sur le bouton 0   temps = 0
Titre_conversationGLOIREOGBONE,2022N2122
prenom : ESSOHANAWE
prenom in Titre_conversation : False
nom : MEWEZINO
nom in Titre_conversation : False
tel : 70111816
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : False
click sur le bouton 1   temps = 0
Titre_conversationGLOIREOGBONE,2022N2122
prenom : ESSOHANAWE
prenom in Titre_conversation : False
nom : MEWEZINO
nom in Titre_conversation : False
tel : 70111816
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : False
click sur le bouton 2   temps = 0
Titre_conversationESSOHANAWEMEW

...

0
1
2
3
4
dans le while recherche de la conversation
click sur le bouton 0   temps = 0
Titre_conversation2021N1804
prenom : 
prenom in Titre_conversation : True
nom : 
nom in Titre_conversation : True
tel : 91070914
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : False
Le titre correspond !
5
Titre conversation : 2021 N1804
Élément non trouvé pour les chemins : ['/html/body/div[1]/div/div/div[2]/div[3]/span/div/div/div[2]/div[1]/div/div/div/div/div/div[2]/div[2]/div[1]/span/span/strong[@class="i0jNr"][text()="Le premier concours d\'entrée"]', '//header[//*[contains(text(),"Rechercher des messages")]]/following-sibling::div//div[@class="_3uIPm WYyr1"]/div[@class="_3m_Xw"]/div/div/div/div/div/span[@class="Hy9nV"][contains(@title, "Le premier concours d\'entrée")]', '//header[//*[contains(text(),"Rechercher des messages")]]/following-sibling::div[@id="pane-side"]/div/div/span[@dir="auto"][@class="i0jNr"][text()="Aucun message trouvé"]']
 | contact trouvé 0

0
1
2
3
4
dans le while recherche de la conversation
click sur le bouton 0   temps = 0
Titre_conversation2021N1820
prenom : 
prenom in Titre_conversation : True
nom : 
nom in Titre_conversation : True
tel : 70588870
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : True
Le titre correspond !
5
Titre conversation : 2021 N1820
 | contact trouvé 0 | 2 | 2.1 | contact sélectionné | message vraiment non trouvé | aucun des mots-clés interdits n'a été trouvé -> envoi




91267670
0
1
2
3
4
dans le while recherche de la conversation
click sur le bouton 0   temps = 0
Titre_conversation2021N1821
prenom : 
prenom in Titre_conversation : True
nom : 
nom in Titre_conversation : True
tel : 91267670
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : True
Le titre correspond !
5
Titre conversation : 2021 N1821
 | contact trouvé 0 | 2 | 2.1 | contact sélectionné | message vraiment non trouvé | aucun des mots-clés interdits n'a été trouvé -> e

0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
90132795
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
92466445
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
92582563
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
93763957
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
91113194
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
96222063
0
1
contact officiellement non trouvé
contact offici

0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
99366381
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
90971715
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
91817907
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
93999118
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
99077895
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
70293860
0
1
contact officiellement non trouvé
contact offici





93066479
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
79876561
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
90717286
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
97768456
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
93893514
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
96924460
0
1
contact officiellement non trouvé
contact officiellement non trouvé
 | Contact officiellement non trouvé | Contact non trouvé
Contact non trouvé
92864208
0
1
contact officiellement non trouvé
c